In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.375
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from geocat.comp import eofunc_eofs, eofunc_pcs
from datetime import datetime
import warnings
from matplotlib.patches import Rectangle
import random
import numba
import statsmodels.api as sm
from sklearn.metrics import r2_score
import pandas as pd
import os
import sys

# Step 1: Add parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "/scratch/ivyglade/pdo/"))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Step 2: Now you can import file.py
import pdo_functions  # assuming file.py contains functions/classes

import importlib
importlib.reload(pdo_functions)

In [ ]:
# Open SST data and compute the PDO and Nino3.4
sst = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/HadISST_sst.nc')['sst']

pdo_1980_2010 = pdo_functions.pdo_from_hadisst(sst, 1980, 2010)

pdo_1870_2024 = pdo_functions.pdo_from_hadisst(sst, 1870, 2024)

In [ ]:
# Convert pdo to xarray
pdo_1980_2010_xr = xr.DataArray(pdo_1980_2010, coords={'time':sst['time']}, dims=['time'])
pdo_1870_2024_xr = xr.DataArray(pdo_1870_2024, coords={'time':sst['time']}, dims=['time'])

# Subset only 1940-2024
pdo_1980_2010_sub = pdo_1980_2010_xr.sel(time=pdo_1980_2010_xr.time.dt.year.isin(np.arange(1940, 2025, 1)))
pdo_1870_2024_sub = pdo_1870_2024_xr.sel(time=pdo_1870_2024_xr.time.dt.year.isin(np.arange(1940, 2025, 1)))

# only MAM and take seasonal averages
pdo_1980_2010_sub_mam = pdo_1980_2010_sub.sel(time=pdo_1980_2010_sub.time.dt.month.isin([3, 4, 5])).resample(time='YE').mean()
pdo_1870_2024_sub_mam = pdo_1870_2024_sub.sel(time=pdo_1870_2024_sub.time.dt.month.isin([3, 4, 5])).resample(time='YE').mean()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))

years = np.arange(1940, 2025, 1)

ax.bar(years[pdo_1980_2010_sub_mam > 0], pdo_1980_2010_sub_mam[pdo_1980_2010_sub_mam > 0], color='xkcd:carmine', label='1980-2010 base period (+) PDO')
ax.bar(years[pdo_1980_2010_sub_mam < 0], pdo_1980_2010_sub_mam[pdo_1980_2010_sub_mam < 0], color='xkcd:sapphire', label='1980-2010 base period (-) PDO')

ax.bar(years[pdo_1870_2024_sub_mam > 0], pdo_1870_2024_sub_mam[pdo_1870_2024_sub_mam > 0], color='xkcd:dirty pink', alpha=0.7, label='1870-2024 base period (+) PDO')
ax.bar(years[pdo_1870_2024_sub_mam < 0], pdo_1870_2024_sub_mam[pdo_1870_2024_sub_mam < 0], color='xkcd:light grey blue', alpha=0.7, label='1870-2024 base period (-) PDO')

ax.legend(frameon=False, fontsize=8, loc='upper left', bbox_to_anchor=(0.1, 1))

ax.set_ylabel('PDO index')
ax.set_xlabel('year')